In [24]:
import csv
import pandas as pd    
from math import sqrt
from numpy.random import normal 
import collections
from numpy.random import choice
import random  
import numpy as np
import time   
import copy

In [40]:
#Initial Setting.

Primary_Data = 'SAPN_0.2_0.4.csv'
Input_Data = 'SAS_0.1.csv'
Initial_Size = 100
Selection_Size = 200
Number_of_Machines = 5
Number_of_Jobs = 5
Number_of_Iteration = 100
pop_size = 200
Num_of_Simu = 40
Num_of_Rep = 20
Crossover_Rate=0.98
Mutation_Rate=0.02
OpperJob_dict = {'J1':5,'J2':5,'J3':5,'J4':5,'J5':5}
OperationDict = Operation_Dict(Primary_Data, Number_of_Machines, Number_of_Jobs)
SetupTimeDict = SetUpTime_Dict(Input_Data, OpperJob_dict)

In [25]:
#The purpose of this function is to read the operation data from the 'SAPN_0.2_0.4.csv' file as input. It then generates a 
#dictionary where each operation serves as a key. For each key within the dictionary, the corresponding values include 
#information about the associated job, the machines capable of performing the operation, the processing times, the 
#sequence of the operation within the job's operations, and whether it is considered a critical operation for the 
#corresponding job or not.

def Operation_Dict(preliminary_data, Number_of_Machines, Number_of_Jobs):   
    
    OperationDict={}
    op_id_counter=1
    machine_set=[]
    sorting_list = []
    df = pd.read_csv(preliminary_data)
    
    for row in range(len(df)):
        OperationDict['Op'+str(op_id_counter)] = {'job': df.iloc[row][0], 
                                          'seq': int(df.iloc[row][1].split('O')[1]),
                                          'machineset':{},
                                          'IsCritical':str(df.loc[row]['IsCritical'])}
        
        for i in range(Number_of_Machines):
            if df.iloc[row, (2*i)+2] != '-':
                OperationDict['Op'+str(op_id_counter)]['machineset']. update( {'M'+str(i+1):{
                                  'Mean':int(df.iloc[row,  (2*i)+2]),
                                  'Var':int(df.iloc[row,  (2*i)+3])
                }} )
                
                                    
        op_id_counter+=1                                        
    for op in OperationDict:
        sorting_list.append([op,OperationDict[op]['job'],OperationDict[op]['seq']])


    sorting_list = sorted(sorting_list, key = lambda x: (x[1], x[2]))

    for i in range(len(sorting_list)):
        if i==0:
            sorting_list[i].append('N/A')
        else:
            if sorting_list[i][1] == sorting_list[i-1][1]:
                sorting_list[i].append(sorting_list[i-1][0])
            else:
                sorting_list[i].append('N/A')
                
    for i in range(len(sorting_list)):
        OperationDict[sorting_list[i][0]]['predecessor'] = sorting_list[i][3]

    return(OperationDict)

In [23]:
#The function is designed to read the sequence-dependent setup time data from the 'SAS_0.1.csv' file as input. 
#It then creates a dictionary where each operation is used as a key. For each key in the dictionary, the mean and 
#variance of the corresponding setup time are recorded.

def SetUpTime_Dict(input_data, OpperJob_dict) :  

    Number_of_Ops=sum(OpperJob_dict.values())
    
    SetupTimeDict={'Op'+ str(i+1):{ } for i in range(Number_of_Ops)}

    op_id_counter=1

    df = pd.read_csv(input_data)
    
    for col in range(1,len(df.columns)):
        for row in range(0,len(df)):
            SetupTimeDict['Op'+str(op_id_counter)].update({'Op'+ str(row+1):{'Mean':float(((df.iloc[row][col]).split(';'))[0]),
                                                                            'Var':float(((df.iloc[row][col]).split(';'))[1])
                                                                             } })
        op_id_counter+=1
        
    return(SetupTimeDict)


In [35]:
#The output of this function is the precedence orders sets defining the execution precedence of operations of the 
#same jobs

def Precedence_Depending_Set(OpperJob_dict): 
    
    Predep_set =()
    Predep_list =()

    Num_of_Ops = sum(OpperJob_dict.values())
    op_counter = 0    
    
    for job in  OpperJob_dict:       
        for op in range (0, OpperJob_dict[job]):
            
                       Predep_set = Predep_set + (('Op' + str(op_counter+1)),)
                       op_counter+=1
                    
        Predep_list = Predep_list + (Predep_set,)
        Predep_set = ()
        
    return(Predep_list)

In [33]:
#The purpose of this function is to generate feasible solutions for the scheduling problem, which are then used to
#form the Initial Population. A feasible solution, in this context, refers to a Job-Shop Queue (Queue) or a chromosome
#of operations where the precedence relationships among the operations of the same jobs are respected.

def feasible_solution(Operation_Dict, Number_of_Jobs):
   

    JobOpDict = {'J'+str(i+1):[] for i in range(Number_of_Jobs)}

    for op in Operation_Dict.keys():
        JobOpDict[Operation_Dict[op]['job']].append((op, Operation_Dict[op]['seq']))

    for job in JobOpDict.keys():
        JobOpDict[job] = sorted(JobOpDict[job], key = lambda x: (x[1]))


    Chromosome = []

    for i in range(len(list(Operation_Dict.keys()))):

        for job in list(JobOpDict.keys()):
            if len(JobOpDict[job])==0:
                JobOpDict.pop(job)

        rand_job = choice(list(JobOpDict.keys()))

        
        Chromosome.append(JobOpDict[rand_job][0][0])
        JobOpDict[rand_job].pop(0)


    return Chromosome

In [34]:
#This function generates an Innitial Population for the first iteration of the GA.

def Intial_Population(Operationdict,Number_of_Machines,Number_of_Jobs,Population_Size):
    Initial_Pop=[]
    for i in range(Population_Size):
        feasiblesolution= feasible_solution (Operationdict,Number_of_Jobs) 
        Initial_Pop.append(feasiblesolution)
    return(Initial_Pop)

In [30]:
##This function represents the selection operator.

def Selection(Operationdict,SetUpTime,InitialPop,selection_size,Number_of_Machines,Number_of_Jobs,
                                                                  Number_of_Simu):
    
    fit_dict = {'Chromozom ' + str(i+1) : [] for i in range(len(InitialPop))}
    Selection_Pop=[]
    score_one = 0
    score_two = 0
    chrom_counter = 0
    for chrom in InitialPop:
        for j in range(Number_of_Simu):
              fit_dict['Chromozom '+ str(chrom_counter+1)].append(Fitness(OperationDict,SetUpTime, Number_of_Machines,
                                       Number_of_Jobs, chrom))
        chrom_counter+=1           
    
    while len(Selection_Pop) != Selection_Size:
        choice_one = random.choice(InitialPop)
        index_one = InitialPop.index(choice_one)
        choice_two = random.choice(InitialPop)
        index_two = InitialPop.index(choice_two)

        for fit in fit_dict['Chromozom ' + str(index_one+1)]:
                    index_fit = fit_dict['Chromozom ' + str(index_one+1)].index(fit)
                
                    if fit <= fit_dict['Chromozom ' + str(index_two+1)][index_fit]:
                          score_one+=1
                    else:
                          score_two+=1
                        
        if score_one == Number_of_Simu:               
                Selection_Pop.append(tuple(choice_one,))
                
        elif score_two == Number_of_Simu:
                Selection_Pop.append(tuple(choice_two,))

        score_one = 0
        score_two = 0

    return(Selection_Pop)

In [28]:
#This function represents the crossover operator.

def Crossover(SelectionPop,OpperJob_dict, Crossover_Rate):
    Selection_Size=len(SelectionPop)
    Number_of_Operations= sum (OpperJob_dict.values())
    PreDep_set=Precedence_Depending_Set(OpperJob_dict)
        
    Crossover_Pop=[]
        
    for i in range(int(Selection_Size*Crossover_Rate)):
            parent1=random.choice(SelectionPop)
            parent2=random.choice(SelectionPop)
            child=[0 for i in range(Number_of_Operations)]

            point=random.randint(0,(Number_of_Operations)-1)
            cross=parent1[point]
    
            for op in PreDep_set:
                   if cross in op:
      
                            for opp in op:
                                child.insert(parent1.index(opp),opp)
                                
            for j in range(Number_of_Operations):
                   if parent2[j] not in child:
                            for i in range(Number_of_Operations):
                                if child[i]==0:
                                        child[i]=parent2[j]
                                        break

            child = [op for op in child if op!=0]
            Crossover_Pop.append (tuple(child,))  
            
    return(Crossover_Pop)

In [29]:
#This function represents the mutation operator.

def Mutation (SelectionPop,OpperJob_dict, Mutation_Rate):

    Selection_Size=len(SelectionPop)
    Number_of_Operations= sum(OpperJob_dict.values())
    PreDep_set=Precedence_Depending_Set(OpperJob_dict)
    Mutation_Pop=[]
    rest=[]  
    gen = []

    
    for i in range(int(Selection_Size*Mutation_Rate)):
        point=random.randint(0,(Number_of_Operations)-1)
        child=list(random.choice(SelectionPop))   
        mutation_point1=child[point] 
        for Seq in PreDep_set:
             if mutation_point1 in Seq:
                index=Seq.index(mutation_point1)
                if index!=0 and index!=(len(Seq))-1:
                    predecessor= Seq[index-1]
                    successor=Seq[index+1]
                elif index==0 and len(Seq)==0:
                    predecessor=Seq[index]
                    successor=Seq[index]
                elif index==0 and len(Seq)!=1:
                    predecessor=Seq[index]
                    successor=Seq[index+1]    
                elif index==(len(Seq))-1:
                    predecessor=Seq[index-1]
                    successor=Seq[index]
              
        mutation_index1=child.index(mutation_point1)   
        Spoint=child.index(predecessor)
        Epoint=child.index(successor)

        
        if Epoint - Spoint <=1 :
                        Mutation_Pop.append(tuple(child,))
        else:                 
                        mutation_index2=random.randint(Spoint+1,Epoint-1)

                        if mutation_index2 > point:
         
                                gen = child[:point]
                                gen+= child[point+1:mutation_index2+1]
                                gen.append(child[point]) 
                                gen+=  child[mutation_index2+1:]
                                Mutation_Pop.append(tuple(gen,))

                        if mutation_index2 == point:
                                Mutation_Pop.append(tuple(child,))
        
                        if mutation_index2 < point:
                
                                gen = child[:mutation_index2]
                                gen.append(child[point])
                                rest= child[mutation_index2:]
                                rest.remove(child[point])
                                gen= gen +  rest
                                Mutation_Pop.append(tuple(gen,))
                        

                        gen = []
                        final_gen=[]     
     
    return(Mutation_Pop)   

In [27]:
#This function is responsible for calculating the fitness function for a given input chromosome. 
#In the case where the objective function represents the system's makespan, it directly calls the Operation_Assignment 
#function. However, when the objective function corresponds to the summation of earliness and tardiness, additional 
#calculations need to be performed within this function.

def Fitness(OperationDict, SetUpTime, Number_of_Machines, Number_of_Jobs, final_chromozom):
                       
    
    fit_value = Operation_Assignment (final_chromozom, OperationDict, SetUpTime, Number_of_Machines, Number_of_Jobs)
    
#If the objective function is the summation of earliness and tradiness:    
    #     tardiness = 0
    #     earliness = 0

    #     for job in CompletionTime_Dict.keys():

    #         if CompletionTime_Dict[job] > Due_Date[job]:
    #             tardiness += (CompletionTime_Dict[job] - Due_Date[job]) * CT
    #         if CompletionTime_Dict[job] < Due_Date[job]:    
    #             earliness += (Due_Date[job] - CompletionTime_Dict[job]) * CE

    #     fit_value = tardiness + earliness
    
    return(fit_value)

In [26]:
#The primary objective of this function is to create a schedule plan by assigning the operations from the 
#Job-Shop Queue (Queue) to the available machines within the manufacturing system.

def Operation_Assignment(feasiblesolution, OperationDict, SetUpTime, Number_of_Machines, Number_of_Jobs):

    Machine_busy = {'M'+ str(i+1) : { 'FreeAt': 0 , 'ProcessedOp': '' } for i in range(Number_of_Machines)}
    PredCtrlOp={'J'+str(i+1):{'Op':'-', 'Machine':'-'} for i in range(Number_of_Jobs)}
    Pred_ProcessTime = {'J'+str(i+1): 0 for i in range(Number_of_Jobs)}
    Completion_Time = {'J'+str(i+1): 0 for i in range(Number_of_Jobs)}
    Assign_Dict = {'M'+str(i+1):[] for i in range(Number_of_Machines)}
    machine_set={}
    CrtlMachineSet=[]
    Counter=0
    Makespan=[]

    for op in feasiblesolution:
        
        Counter=0
        CrtlMachineSet=[]        
        Job =  OperationDict[op]['job'] 
        Pred = PredCtrlOp[Job]['Op']
        Crtl= OperationDict[op]['IsCritical']
       
        if Crtl == 'True' and Pred != '-' :
            
                Machine =  PredCtrlOp [Job]['Machine']
                PredCtrlOp[Job]['Op'] = op
                PredCtrlOp [Job]['Machine'] = Machine
                
        elif  Crtl == 'True' and Pred == '-':   

                for opp in OperationDict.keys():
                    if OperationDict[opp]['job'] == Job and OperationDict[opp]['IsCritical'] == 'True':
                        Counter+=1 
                        CrtlMachineSet = CrtlMachineSet+list(OperationDict[opp]['machineset'].keys())
                CrtlMachineSet=[m for m, count in collections.Counter(CrtlMachineSet).items() if count == Counter]
                    
                for machine in CrtlMachineSet:
                    machine_set[machine]=Machine_busy[machine]['FreeAt']
                        
                Machine=min(machine_set, key=machine_set.get)    
                PredCtrlOp[Job]['Op'] = op
                PredCtrlOp [Job]['Machine'] = Machine
                machine_set.clear() 
                
        elif  Crtl == 'False':
            
                for machine in OperationDict[op]['machineset']:
                    
                    machine_set[machine]=Machine_busy[machine]['FreeAt']
                Machine=min(machine_set, key=machine_set.get)              
                machine_set.clear() 
                
        Mean = int(OperationDict[op]['machineset'][Machine]['Mean'])
        Std = int(sqrt(OperationDict[op]['machineset'][Machine]['Var']))
        ProcessTime = float(normal(Mean, Std, 1))  
        Previous_Op = Machine_busy[Machine]['ProcessedOp']

        if Previous_Op != '':
                MeanST = int(SetUpTime[op][Previous_Op]['Mean'])
                StdST = float(SetUpTime[op][Previous_Op]['Var'])
                StpTime = float(normal(MeanST, StdST, 1))  
        else:
                StpTime=0
            
        StartTime = max(Machine_busy[machine]['FreeAt']+StpTime , Pred_ProcessTime[Job], 0)

        Assign_Dict[Machine].append({ 'operation': op,
                                      'job': Job,
                                      'startTime': StartTime,
                                      'SetUpTime' : StpTime,
                                      'endTime': StartTime + ProcessTime,
                                      'duration': ProcessTime
                                    })
        Completion_Time[Job]+= ProcessTime + StpTime
        Machine_busy[Machine]['FreeAt'] = StartTime + ProcessTime
        Machine_busy[Machine]['ProcessedOp'] = op
        Pred_ProcessTime[Job] = StartTime + ProcessTime


    for machine in Machine_busy:
        Makespan.append(Machine_busy[machine]['FreeAt'])
    MakeSpan= max (Makespan) 
    return(MakeSpan) 

In [36]:
# This function follows a similar procedure as the Operation_Assignment function. However, in this case, the output is 
# a dictionary where each machine serves as a key. The corresponding values for each key are the operations that have 
# been assigned to that particular machine.

def Assignment_Dict(feasiblesolution, Operation_Dict, SetUpTime, Number_of_Machines, Number_of_Jobs):

    Machine_busy = {'M'+ str(i+1) : { 'FreeAt': 0 , 'ProcessedOp': '' } for i in range(Number_of_Machines)}
    PredCtrlOp={'J'+str(i+1):{'Op':'-', 'Machine':'-'} for i in range(Number_of_Jobs)}
    Pred_ProcessTime = {'J'+str(i+1): 0 for i in range(Number_of_Jobs)}
    Completion_Time = {'J'+str(i+1): 0 for i in range(Number_of_Jobs)}
    Assign_Dict = {'M'+str(i+1):{'operation'} for i in range(Number_of_Machines)}
    machine_set={}
    CrtlMachineSet=[]
    Counter=0
    Makespan=[]

    for op in feasiblesolution:
        
        Counter=0
        CrtlMachineSet=[]        
        Job =  OperationDict[op]['job'] 
        Pred = PredCtrlOp[Job]['Op']
        Crtl= OperationDict[op]['IsCritical']
       
        if Crtl == 'True' and Pred != '-' :
            
                Machine =  PredCtrlOp [Job]['Machine']
                PredCtrlOp[Job]['Op'] = op
                PredCtrlOp [Job]['Machine'] = Machine
                
        elif  Crtl == 'True' and Pred == '-':   

                for opp in OperationDict.keys():
                    if OperationDict[opp]['job'] == Job and OperationDict[opp]['IsCritical'] == 'True':
                        Counter+=1 
                        CrtlMachineSet = CrtlMachineSet+list(OperationDict[opp]['machineset'].keys())
                CrtlMachineSet=[m for m, count in collections.Counter(CrtlMachineSet).items() if count == Counter]
                    
                for machine in CrtlMachineSet:
                    machine_set[machine]=Machine_busy[machine]['FreeAt']
                        
                Machine=min(machine_set, key=machine_set.get)    
                PredCtrlOp[Job]['Op'] = op
                PredCtrlOp [Job]['Machine'] = Machine
                machine_set.clear() 
                
        elif  Crtl == 'False':
            
                for machine in OperationDict[op]['machineset']:
                    
                    machine_set[machine]=Machine_busy[machine]['FreeAt']
                    
                Machine=min(machine_set, key=machine_set.get)              
                machine_set.clear() 

        
        Mean = int(OperationDict[op]['machineset'][Machine]['Mean'])
        Std = int(sqrt(OperationDict[op]['machineset'][Machine]['Var']))
        ProcessTime = float(normal(Mean, Std, 1))  
        Previous_Op = Machine_busy[Machine]['ProcessedOp']

        if Previous_Op != '':
                MeanST = int(SetUpTime[op][Previous_Op]['Mean'])
                StdST = int(SetUpTime[op][Previous_Op]['Var'])
                StpTime = float(normal(MeanST, StdST, 1))  
        else:
                StpTime=0
            
        StartTime = max(Machine_busy[machine]['FreeAt']+StpTime , Pred_ProcessTime[Job], 0)

        Assign_Dict[Machine].update({op})
       
        Completion_Time[Job]+= ProcessTime + StpTime
        Machine_busy[Machine]['FreeAt'] = StartTime + ProcessTime
        Machine_busy[Machine]['ProcessedOp'] = op
        Pred_ProcessTime[Job] = StartTime + ProcessTime

    return(Assign_Dict)

In [22]:
#This Function is in charge of creating the Next Population within the GA structure.

def NonDominant_Dict(Operationdict,SetUpTime,Number_of_Machines, Number_of_Jobs,GenSum, Num_of_Simu):
        
    p=0.1   
    fit_dict = {'Chromozom ' + str(i+1) :{'Fitness':[], 'CompareNum': 0, 'DominatedbyNum': 0,'Dominates':[], 'Rank':0  } 
                for i in range(int(len(GenSum)))} 

    chrom_dict= {'Chromozom ' + str(i+1) : []  for i in range(int(len(GenSum)))}
    crowd_dict= {'Chromozom ' + str(i+1) : 0  for i in range(int(len(GenSum)))}
    chrom_counter = 1
    list_of_fitness_values = []
    chrom_num = 0
######################################################################    
    for i in range(Num_of_Simu):
        for chrom  in GenSum: 
                chrom_dict['Chromozom '+str(chrom_counter)] = chrom
                fit = Fitness(Operationdict, SetUpTime, Number_of_Machines,Number_of_Jobs, 
                              chrom)
                fit_dict['Chromozom ' + str(chrom_counter)]['Fitness'].append(fit)
                list_of_fitness_values.append(('Chromozom ' + str(chrom_counter), fit))
                chrom_counter+=1   
        chrom_counter=1  
    a = int(len(list_of_fitness_values)/2)   
#     print(a)
    list_of_fitness_values = sorted(list_of_fitness_values, key=lambda x:x[1])
    for chrom in fit_dict:
        for i in range(a):
            if list_of_fitness_values[i][0] == chrom:
                fit_dict[chrom]['CompareNum'] += 1
        
    
    compare_dict = { i : []  for i in range(int(len(list_of_fitness_values)))}
    for chrom in fit_dict.keys():
        compare_dict[fit_dict[chrom]['CompareNum']].append(chrom)
####################################################################            
    for k in list(compare_dict.keys()):
        if len(compare_dict[k]) == 0:
            compare_dict.pop(k)
            
    keylist = compare_dict.keys()
    keylist = sorted(keylist, reverse = True)
    next_gen = []
    NonDominanSolution = []
    chrom_assignmmentdict = {'Chromozom ' + str(i+1) : {'M'+str(j+1):{'operation'} for j in range(Number_of_Machines)}
                                                         for i in range(int(len(GenSum)))}

    while len(next_gen) != pop_size:
        
         for comparenum in keylist:
                if len(compare_dict[comparenum] + next_gen) <= pop_size:
                        next_gen += compare_dict[comparenum]

                elif len(compare_dict[comparenum] + next_gen) > pop_size:
                        for chromozom in compare_dict[comparenum]:
                            
                                    assignmentdict = Assignment_Dict(chrom_dict[chromozom], OperationDict, SetUpTime, Number_of_Machines, Number_of_Jobs)
                                                                         
                                    for machine in assignmentdict.keys():
                                            chrom_assignmmentdict[chromozom][machine]=(assignmentdict[machine])  
         
                        for solution in compare_dict[comparenum]:
                                    crowd_dict[solution]=0
                        for solution in compare_dict[comparenum]:
                                    index = compare_dict[comparenum].index(solution)
                               
                                    for other in compare_dict[comparenum][index+1:]:
                                                if chrom_assignmmentdict[solution]==chrom_assignmmentdict[other]:
                                
                                                        crowd_dict[solution]+=1
                                                        crowd_dict[other]+=1
                                        
                        subtract = pop_size - len(next_gen) 
                        sort= dict (sorted (crowd_dict.items(), key=lambda item: item[1]))
                        next_gen+=list(sort.keys())[:subtract]  
                        chrom_assignmmentdict.clear()
                        crowd_dict.clear()
                        break
###############################################                        
    for gen in next_gen:
        NonDominanSolution.append (tuple(chrom_dict[gen]),)

    return(NonDominanSolution)                              


In [32]:
#This function serves as the integration point for all the components of the Genetic Algorithm (GA). 
#It combines and coordinates all the different elements of the GA within a single function.

def Genetic_Algorithm(OperationDict, SetUpTime, Initial_Size, Selection_Size, Number_of_Machines, 
                       Number_of_Jobs, OpperJob_dict, Number_of_Iteration, Num_of_Simu,Crossover_Rate, Mutation_Rate):
    
    ChromozomSet_one = Intial_Population(OperationDict, Number_of_Machines, Number_of_Jobs, Initial_Size)
    
    IterFit={}
    IterChrom={}
    IterFitList1=[]
    IterFitList2=[]
    IterFinish_time= []
    start_time = time.time()
    for i in range(Number_of_Iteration):
        
            ThisIterStart_time = time.time()
            fit1=[]
            fit2=[]
            
            for Chrome in ChromozomSet_one:
                
                fitness1 = Fitness(OperationDict,SetUpTime, Number_of_Machines,Number_of_Jobs,
                                                    Chrome)
                
                fit1.append(fitness1)
                IterFitList1.append((Chrome,fitness1))

                    
            Sel= Selection(OperationDict,SetUpTime, ChromozomSet_one,Selection_Size, Number_of_Machines,Number_of_Jobs, Num_of_Simu)


            Cross = Crossover(Sel, OpperJob_dict, Crossover_Rate)

            Mute = Mutation(Sel, OpperJob_dict, Mutation_Rate)

            GenSum=Cross+Mute+ChromozomSet_one
            ChromozomSet_two = (NonDominant_Dict(OperationDict, SetUpTime, Number_of_Machines, Number_of_Jobs, 
                                                                                       GenSum, Num_of_Simu))
            

            EndofIter_time = time.time()
            IterFinish_time . append(EndofIter_time - start_time)
            
            if EndofIter_time - start_time >= 600:
                
                print(f"\t\t\t\t Runtime in Seconds: {IterFinish_time[i-1]}\n")
                break
                
            for Chrome in ChromozomSet_two:
                    fitness2= (Fitness(OperationDict, SetUpTime, Number_of_Machines,Number_of_Jobs,
                                                                       Chrome))

                    fit2.append (fitness2)
                    IterFitList2.append((Chrome,fitness2))

                    IterFit['Iteration'+ str(i+1)] = min (fit1)
                    for sett in IterFitList1:
                              if sett[1]  == min (fit1):
                    
                                     IterChrom['Iteration'+ str(i+1)]= sett[0]

                    IterFit['Iteration'+ str(i+1)] = min (fit2)
                    for sett in IterFitList2:
                             if sett[1]  == min (fit2):
                                         IterChrom['Iteration'+ str(i+1)]= sett[0]

                               
                    ChromozomSet_one=ChromozomSet_two  
                    
            Iter = 'Iteration '+ str(i+1)
            print(' %s%f'%(Iter + '\n Fitness : ',IterFit['Iteration'+ str(i+1)]))

            print(120*'-')
        
       
    if i == Number_of_Iteration-1:
        print(f"\t\t\t\t Runtime in Seconds: {IterFinish_time[i]}\n")
       
    return(IterFit, IterChrom)     


In [31]:
#This function is responsible for executing the simulation model for SL = 10^5 times for the best solution obtained 
#in each replication. It then calculates and reports the mean and variance of the fitness values obtained in the 
#simulation runs.

def Comparison (OperationDict,SetUpTime, Number_of_Machines,Number_of_Jobs,
                        final_chromozom):
    
    fit_list = []
    for i in range(100000):
        fit_list.append(Fitness(OperationDict,SetUpTime, Number_of_Machines,Number_of_Jobs,
                        final_chromozom))
        
    Mean = np.mean(fit_list)
    STD = sqrt(np.var(fit_list))

    return(Mean, STD)


In [37]:
# After executing all the above code cells, this function should be called, which provides a step-by-step report of 
#the results obtained from the Simulation-Optimization (SO) model.

def Result(OperationDict, SetUpTime, Initial_Size, Selection_Size, Number_of_Machines, Number_of_Jobs, OpperJob_dict, 
                                     Number_of_Iteration, Num_of_Simu, Num_of_Rep,Crossover_Rate,Mutation_Rate):
    start_tot = time.time()
    mean_list = []
    std_list = []
    chrom_list = []
    
    for i in range(Num_of_Rep):
        
                start = time.time()       
                        
                print('%s%s'%('\n Replication ' +str(i+1),'\n'))
                final_fit, final_set  =  Genetic_Algorithm(OperationDict, SetUpTime,Initial_Size, Selection_Size, Number_of_Machines,
                                     Number_of_Jobs, OpperJob_dict, Number_of_Iteration, Num_of_Simu,Crossover_Rate,Mutation_Rate)
                end=time.time()
 
                best_fit = sorted (final_fit.items(), key=lambda item: item[1])[0][1]
                final_iter = sorted (final_fit.items(), key=lambda item: item[1])[0][0]

                Rep_Mean, Rep_STD = Comparison (OperationDict, SetUpTime, Number_of_Machines,Number_of_Jobs,
                                                final_set[final_iter])

                mean_list.append(Rep_Mean)
                std_list.append(Rep_STD)
                
                print('%s%f'%('\t\t\t  The Best Fitness of this Replication: ', best_fit))
                print('%s%f%s%f'%('\n\t\t\t\t   Mean: ', Rep_Mean, '\t  STD: ',Rep_STD))
                
                chrom_list.append((final_set[final_iter], Rep_Mean))


    Mean = np.mean(mean_list)
    STD = np.mean(std_list)
    print(120*'#')

    print('\n%s%f%s%f'%('\n\t\t\t  Mean of 20 Replications: ', Mean, '\t  STD of 20 Replications:',STD))
    chrom_list.sort(key=lambda x: x[1])
    Best_Chrome = chrom_list[0][0]
    print(120*'#')
    end_tot=time.time()

In [41]:
Result(OperationDict,SetupTimeDict, Initial_Size, Selection_Size, Number_of_Machines,
           Number_of_Jobs, OpperJob_dict, Number_of_Iteration, Num_of_Simu, Num_of_Rep,Crossover_Rate,Mutation_Rate)                               



 Replication 1

 Iteration 1
 Fitness : 20.919347
------------------------------------------------------------------------------------------------------------------------
 Iteration 2
 Fitness : 20.630191
------------------------------------------------------------------------------------------------------------------------
 Iteration 3
 Fitness : 21.891341
------------------------------------------------------------------------------------------------------------------------
 Iteration 4
 Fitness : 21.233981
------------------------------------------------------------------------------------------------------------------------
 Iteration 5
 Fitness : 20.852744
------------------------------------------------------------------------------------------------------------------------
 Iteration 6
 Fitness : 22.069974
------------------------------------------------------------------------------------------------------------------------
 Iteration 7
 Fitness : 20.614935
-------------------

 Iteration 23
 Fitness : 21.602267
------------------------------------------------------------------------------------------------------------------------
 Iteration 24
 Fitness : 24.600110
------------------------------------------------------------------------------------------------------------------------
 Iteration 25
 Fitness : 24.678806
------------------------------------------------------------------------------------------------------------------------
 Iteration 26
 Fitness : 24.917690
------------------------------------------------------------------------------------------------------------------------
 Iteration 27
 Fitness : 24.663697
------------------------------------------------------------------------------------------------------------------------
 Iteration 28
 Fitness : 22.343339
------------------------------------------------------------------------------------------------------------------------
 Iteration 29
 Fitness : 22.646468
-----------------------------

 Iteration 12
 Fitness : 20.113180
------------------------------------------------------------------------------------------------------------------------
 Iteration 13
 Fitness : 21.113465
------------------------------------------------------------------------------------------------------------------------
 Iteration 14
 Fitness : 20.613982
------------------------------------------------------------------------------------------------------------------------
 Iteration 15
 Fitness : 20.214161
------------------------------------------------------------------------------------------------------------------------
 Iteration 16
 Fitness : 21.380393
------------------------------------------------------------------------------------------------------------------------
 Iteration 17
 Fitness : 20.619901
------------------------------------------------------------------------------------------------------------------------
 Iteration 18
 Fitness : 20.421618
-----------------------------

 Iteration 1
 Fitness : 21.332904
------------------------------------------------------------------------------------------------------------------------
 Iteration 2
 Fitness : 21.105485
------------------------------------------------------------------------------------------------------------------------
 Iteration 3
 Fitness : 22.041744
------------------------------------------------------------------------------------------------------------------------
 Iteration 4
 Fitness : 22.070882
------------------------------------------------------------------------------------------------------------------------
 Iteration 5
 Fitness : 23.680408
------------------------------------------------------------------------------------------------------------------------
 Iteration 6
 Fitness : 24.303669
------------------------------------------------------------------------------------------------------------------------
 Iteration 7
 Fitness : 23.492703
------------------------------------

 Iteration 20
 Fitness : 24.126207
------------------------------------------------------------------------------------------------------------------------
 Iteration 21
 Fitness : 24.077396
------------------------------------------------------------------------------------------------------------------------
 Iteration 22
 Fitness : 22.680606
------------------------------------------------------------------------------------------------------------------------
 Iteration 23
 Fitness : 23.873391
------------------------------------------------------------------------------------------------------------------------
 Iteration 24
 Fitness : 23.900920
------------------------------------------------------------------------------------------------------------------------
 Iteration 25
 Fitness : 24.552454
------------------------------------------------------------------------------------------------------------------------
 Iteration 26
 Fitness : 24.640721
-----------------------------

...

 Iteration 25
 Fitness : 21.472070
------------------------------------------------------------------------------------------------------------------------
 Iteration 26
 Fitness : 21.124340
------------------------------------------------------------------------------------------------------------------------
 Iteration 27
 Fitness : 22.243433
------------------------------------------------------------------------------------------------------------------------
 Iteration 28
 Fitness : 23.848193
------------------------------------------------------------------------------------------------------------------------
 Iteration 29
 Fitness : 23.111211
------------------------------------------------------------------------------------------------------------------------
 Iteration 30
 Fitness : 23.383358
------------------------------------------------------------------------------------------------------------------------
 Iteration 31
 Fitness : 22.317682
-----------------------------

 Iteration 14
 Fitness : 24.428969
------------------------------------------------------------------------------------------------------------------------
 Iteration 15
 Fitness : 23.592508
------------------------------------------------------------------------------------------------------------------------
 Iteration 16
 Fitness : 23.217744
------------------------------------------------------------------------------------------------------------------------
 Iteration 17
 Fitness : 24.496530
------------------------------------------------------------------------------------------------------------------------
 Iteration 18
 Fitness : 24.461285
------------------------------------------------------------------------------------------------------------------------
 Iteration 19
 Fitness : 23.893935
------------------------------------------------------------------------------------------------------------------------
 Iteration 20
 Fitness : 23.578913
-----------------------------

 Iteration 4
 Fitness : 21.277338
------------------------------------------------------------------------------------------------------------------------
 Iteration 5
 Fitness : 20.593741
------------------------------------------------------------------------------------------------------------------------
 Iteration 6
 Fitness : 24.835099
------------------------------------------------------------------------------------------------------------------------
 Iteration 7
 Fitness : 21.035332
------------------------------------------------------------------------------------------------------------------------
 Iteration 8
 Fitness : 23.076083
------------------------------------------------------------------------------------------------------------------------
 Iteration 9
 Fitness : 23.063939
------------------------------------------------------------------------------------------------------------------------
 Iteration 10
 Fitness : 24.748182
-----------------------------------

 Iteration 25
 Fitness : 23.059631
------------------------------------------------------------------------------------------------------------------------
				 Runtime in Seconds: 557.4966099262238

			  The Best Fitness of this Replication: 19.149078

				   Mean: 28.594324	  STD: 3.666784

 Replication 16

 Iteration 1
 Fitness : 21.783828
------------------------------------------------------------------------------------------------------------------------
 Iteration 2
 Fitness : 22.425093
------------------------------------------------------------------------------------------------------------------------
 Iteration 3
 Fitness : 21.186984
------------------------------------------------------------------------------------------------------------------------
 Iteration 4
 Fitness : 21.635450
------------------------------------------------------------------------------------------------------------------------
 Iteration 5
 Fitness : 21.459507
-----------------------------------

 Iteration 14
 Fitness : 23.579028
------------------------------------------------------------------------------------------------------------------------
 Iteration 15
 Fitness : 24.566263
------------------------------------------------------------------------------------------------------------------------
 Iteration 16
 Fitness : 25.247429
------------------------------------------------------------------------------------------------------------------------
 Iteration 17
 Fitness : 23.275606
------------------------------------------------------------------------------------------------------------------------
 Iteration 18
 Fitness : 24.766194
------------------------------------------------------------------------------------------------------------------------
				 Runtime in Seconds: 565.803049325943

			  The Best Fitness of this Replication: 20.483006

				   Mean: 31.209623	  STD: 3.762062

 Replication 19

 Iteration 1
 Fitness : 22.375693
--------------------------------